In [1]:
!wget http://www.josiahwang.com/dataset/leedsbutterfly/leedsbutterfly_dataset_v1.0.zip

--2021-01-19 11:31:53--  http://www.josiahwang.com/dataset/leedsbutterfly/leedsbutterfly_dataset_v1.0.zip
Resolving www.josiahwang.com (www.josiahwang.com)... 143.95.240.28
Connecting to www.josiahwang.com (www.josiahwang.com)|143.95.240.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 476389461 (454M) [application/zip]
Saving to: ‘leedsbutterfly_dataset_v1.0.zip’

leedsbutterfly_data 100%[===================>] 454.32M  69.7MB/s    in 6.7s    

2021-01-19 11:32:00 (67.4 MB/s) - ‘leedsbutterfly_dataset_v1.0.zip’ saved [476389461/476389461]



In [2]:
!unzip leedsbutterfly_dataset_v1.0.zip

Archive:  leedsbutterfly_dataset_v1.0.zip
   creating: leedsbutterfly/descriptions/
  inflating: leedsbutterfly/descriptions/001.txt  
  inflating: leedsbutterfly/descriptions/002.txt  
  inflating: leedsbutterfly/descriptions/003.txt  
  inflating: leedsbutterfly/descriptions/004.txt  
  inflating: leedsbutterfly/descriptions/005.txt  
  inflating: leedsbutterfly/descriptions/006.txt  
  inflating: leedsbutterfly/descriptions/007.txt  
  inflating: leedsbutterfly/descriptions/008.txt  
  inflating: leedsbutterfly/descriptions/009.txt  
  inflating: leedsbutterfly/descriptions/010.txt  
   creating: leedsbutterfly/images/
  inflating: leedsbutterfly/images/0010001.png  
  inflating: leedsbutterfly/images/0010002.png  
  inflating: leedsbutterfly/images/0010004.png  
  inflating: leedsbutterfly/images/0010005.png  
  inflating: leedsbutterfly/images/0010006.png  
  inflating: leedsbutterfly/images/0010007.png  
  inflating: leedsbutterfly/images/0010008.png  
  inflating: leedsbutterfly

In [31]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [32]:
categories = []
filenames = os.listdir("/content/leedsbutterfly/images")
for filename in filenames:
        category = filename.split(".")[0]
        categories.append(category[0:3])
        
print(categories[0:3])

['007', '005', '001']


In [33]:
df = pd.DataFrame()
df['Image'] = filenames
df['Category'] = categories    

df.head()

,Image,Category
0,0070007.png,007
1,0050090.png,005
2,0010037.png,001
3,0100025.png,010
4,0040159.png,004


In [34]:
df['Category'].value_counts()

006    100
002     93
009     90
004     90
007     89
005     88
010     84
001     82
003     61
008     55
Name: Category, dtype: int64

In [35]:
import cv2
data=[]
labels=[]
filenames = os.listdir("/content/leedsbutterfly/images")
for filename in filenames:
    label = int(str(filename)[:3])
    image = cv2.imread('/content/leedsbutterfly/images/'+filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    labels.append(label)
data = np.array(data) /255

In [36]:
data.shape

(832, 224, 224, 3)

In [37]:
labels = np.array(labels)

In [38]:
labels = labels.reshape(len(labels),1)

In [39]:
labels.shape

(832, 1)

In [40]:
labels = OneHotEncoder().fit_transform(np.asarray(labels).reshape(len(labels),1))

In [41]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.2, random_state=42)

In [42]:
trainAug = ImageDataGenerator(rotation_range=15,fill_mode="nearest")

In [43]:
baseModel = Xception(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(10, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
    layer.trainable = False

83689472/83683744 [==============================] - 0s 0us/step


In [44]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [45]:
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr=1e-3),
    metrics=["accuracy"])
H = model.fit_generator(
    trainAug.flow(trainX, trainY.toarray(), batch_size=8),
    steps_per_epoch=len(trainX) // 8,
    validation_data=(testX, testY.toarray()),
    validation_steps=len(testX) // 8,
    epochs=25)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
83/83 [==============================] - 12s 113ms/step - loss: 1.7117 - accuracy: 0.4161 - val_loss: 0.3524 - val_accuracy: 0.9521
Epoch 2/25
83/83 [==============================] - 7s 86ms/step - loss: 0.5578 - accuracy: 0.8338
Epoch 3/25
83/83 [==============================] - 7s 86ms/step - loss: 0.3541 - accuracy: 0.8971
Epoch 4/25
83/83 [==============================] - 7s 86ms/step - loss: 0.2991 - accuracy: 0.9105
Epoch 5/25
83/83 [==============================] - 7s 86ms/step - loss: 0.2256 - accuracy: 0.9358
Epoch 6/25
83/83 [==============================] - 7s 85ms/step - loss: 0.1755 - accuracy: 0.9601
Epoch 7/25
83/83 [==============================] - 7s 85ms/step - loss: 0.1902 - accuracy: 0.9464
Epoch 8/25
83/83 [==============================] - 7s 85ms/step - loss: 0.1431 - accuracy: 0.9657
Epoch 9/25
83/83 [==============================] - 7s 86ms/step - loss: 0.1389 - accuracy: 0.9580
Epoch 10/25
83/83 [==============================] - 7s 86ms/step

In [47]:
y_pred = model.predict(testX)

In [48]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([7, 1, 3, 5, 5, 9, 4, 3, 2, 5, 7, 6, 3, 1, 1, 6, 0, 9, 4, 9, 7, 8,
       0, 2, 9, 5, 6, 2, 3, 5, 4, 8, 8, 9, 1, 3, 4, 3, 9, 5, 5, 4, 2, 7,
       3, 3, 6, 3, 4, 0, 6, 0, 5, 9, 8, 4, 8, 9, 5, 5, 2, 4, 3, 9, 3, 1,
       3, 9, 5, 6, 1, 7, 6, 4, 8, 8, 1, 6, 3, 3, 9, 2, 7, 8, 0, 1, 4, 9,
       8, 2, 4, 7, 3, 4, 5, 5, 7, 2, 9, 0, 5, 5, 6, 4, 7, 5, 6, 6, 3, 6,
       7, 7, 6, 6, 2, 2, 2, 8, 6, 0, 2, 6, 8, 8, 8, 7, 4, 4, 9, 2, 7, 1,
       1, 7, 6, 0, 6, 7, 8, 5, 0, 1, 6, 3, 1, 3, 5, 8, 0, 5, 3, 9, 3, 5,
       6, 2, 3, 3, 0, 0, 9, 1, 1, 5, 4, 6, 6])

In [54]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(testY.toarray().argmax(axis=1), y_pred)
print('Accuracy :',acc)

Accuracy : 0.9880239520958084
